In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Load dataset
file_path = "/content/drive/MyDrive/my data scineproject/breast-cancer.csv"
df = pd.read_csv(file_path)

# Encode categorical columns using Label Encoding
label_encoders = {}
for col in df.columns:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

# Splitting dataset into features (X) and target (y)
X = df.drop(columns=['class'])  # Features
y = df['class']  # Target

# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Splitting into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply Data Augmentation (SMOTE + Undersampling)
smote = SMOTE(random_state=42)
under_sampler = RandomUnderSampler(random_state=42)
pipeline = Pipeline([('smote', smote), ('under', under_sampler)])
X_train, y_train = pipeline.fit_resample(X_train, y_train)

# Initialize models with Expanded Hyperparameter Tuning
param_grid_rf = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
param_grid_svm = {
    'C': [0.1, 1, 10, 50],
    'kernel': ['rbf', 'linear', 'poly'],
    'gamma': ['scale', 'auto']
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, n_jobs=-1)
svm_grid = GridSearchCV(SVC(probability=True), param_grid_svm, cv=5, n_jobs=-1)

# Train models
rf_grid.fit(X_train, y_train)
svm_grid.fit(X_train, y_train)

# Best models
rf_best = rf_grid.best_estimator_
svm_best = svm_grid.best_estimator_

# Create Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('Random Forest', rf_best),
    ('SVM', svm_best)
], voting='soft')
voting_clf.fit(X_train, y_train)

# Evaluate models
models = {"Random Forest": rf_best, "SVM": svm_best, "Voting Classifier": voting_clf}
results = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

# Display best model
best_model_name = max(results, key=results.get)
print(f"Best Model: {best_model_name} with Accuracy: {results[best_model_name]:.4f}")


Random Forest Accuracy: 0.6897
              precision    recall  f1-score   support

           0       0.77      0.80      0.79        41
           1       0.47      0.41      0.44        17

    accuracy                           0.69        58
   macro avg       0.62      0.61      0.61        58
weighted avg       0.68      0.69      0.68        58

SVM Accuracy: 0.6724
              precision    recall  f1-score   support

           0       0.75      0.80      0.78        41
           1       0.43      0.35      0.39        17

    accuracy                           0.67        58
   macro avg       0.59      0.58      0.58        58
weighted avg       0.66      0.67      0.66        58

Voting Classifier Accuracy: 0.7069
              precision    recall  f1-score   support

           0       0.76      0.85      0.80        41
           1       0.50      0.35      0.41        17

    accuracy                           0.71        58
   macro avg       0.63      0.60      0.